In [1]:
import re
from collections import defaultdict
import pandas as pd
from tqdm import  tqdm
from threading import Thread as th
from time import time

In [3]:
def web(w):
    w = re.sub(r'www.', '', w)
    w = re.sub('https?://', '', w)
    return re.sub('/.*', '', w)

web('https://www.hello.com/world/yeah')

'hello.com'

#### To get list of words in college name

In [4]:
to_remove = ['gov\S*\s?', 'univ\S*\s?', ',*' , 'coll\S*\s?']
pattern = re.compile('|'.join(to_remove))
def col_name(colg):
    colg = re.sub('-|_|&', '', colg)
    colg = re.sub('of |and |inst\S*\s?', '', colg)
    colg = re.sub('eng\S*\s', 'engineering ', colg)
    colg = re.sub('tech\S*\s', 'technology ', colg)
    colg = pattern.subn('', colg)[0].lower().split(' ')
    return list(filter(None, colg))

col_name('rimt- institute of engg. and technology, mandi gobindgarh')

['rimt', 'engineering', 'technology', 'mandi', 'gobindgarh']

In [5]:
def lower(s):
    return s.lower()

In [6]:
def same(row1, row2):
    points = 0
    
#     try:
    col1 = row1['name_list']
    col2 = row2['Name_list']
    if col1[0] == col2[0]:
        points += 2
    points = len(set(col1).intersection(col2)) / len(set(col1).union(col2))  * 5 + points
    points = min(6, points)
#     except:
#         print(points)

#     try:
    if not row2['City'] == 'nan':
        if row1['city'][:4].lower() == row2['City'][:4].lower(): points += 5
#     except:
#         print('city')

#     try:
    if not row1['state'] == 'nan':
        if row1['state'][:min(4, len(row1['state']))].lower() == row2['State'][:min(4, len(row2['State']))].lower(): points += 2
#     except:
#         print('state')

#     try:
    if not row2['Estd'] == 'nan' and not row1['estb'] == 'nan': 
        if row1['estb'] == row2['Estd']: points += 4
#     except:
#         print('estd')

#     try:
    if not row1['website'] == 'nan' and not row2['website'] == 'nan':
        if row1['website'] == row2['website']: points += 4
#     except:
#         print('web')

    if points > 14: return (True, points)
    else: return (False, points)

In [8]:
def mapper(start, end):
    global mapp, remark
    start_time = time()
    
    c360 = pd.read_csv('../all_colleges_car360.csv').astype(str)
    me = pd.read_csv('../all_colleges_mereexams.csv').astype(str)[start:end]
    c360[['State', 'City', 'Name']] = c360[['State', 'City', 'Name']].applymap(lower)
    me[['state', 'city', 'name']] = me[['state', 'city', 'name']].applymap(lower)
    c360.sort_values(['Name'], inplace = True)
    me.sort_values(['name'], inplace = True)
    c360['website'] = c360['website'].apply(web)
    me['website'] = me['website'].apply(web)
    print('Halfway of data loading')
    c360['Name_list'] = c360['Name'].apply(col_name)
    me['name_list'] = me['name'].apply(col_name)
    me['estb'] = me['year_of_establishment']
    me.drop(['year_of_establishment'], axis = 1, inplace=True)
    
    end_time = time()
    print(f'data load completed in {end_time-start_time} s')
    for index, row1 in tqdm(me.iterrows()):
        points = 0
        for index2, row2 in c360.iterrows():
            if matched[index2]: continue
            (match, p)  = same(row1, row2)
            if match: 
                mapp.append([row1['name'] ,row2['Name']])
                matched[index2] = True
                break
            elif p > points:
                points = p
                name = row2['Name']
        if points > 11:
            remark.append([row1['name'] , name])

In [ ]:
t = [0] * 9
mapp = []
remark = []
matched = [False] * 18465

for i in range(8):
    t[i] = th(target = mapper, args = (i*5000, (i+1) * 5000))
t[8] = th(target = mapper, args = (40000, 42908))

for i in range(8):
    t[i].start()
t[8].start()

for i in range(8):
    t[i].join()
t[8].join()
print('script successful')

In [9]:
mapp = []
remark = []
matched = [False] * 18465
mapper(100, 150)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Halfway of data loading


0it [00:00, ?it/s]

data load completed in 1.6919834613800049 s


50it [03:32,  4.25s/it]


In [1]:
remark

NameError: name 'remark' is not defined

In [12]:
c360 = pd.read_csv('../all_colleges_car360.csv').astype(str)
me = pd.read_csv('../all_colleges_mereexams.csv').astype(str)[start:end]
c360[['State', 'City', 'Name']] = c360[['State', 'City', 'Name']].applymap(lower)
me[['state', 'city', 'name']] = me[['state', 'city', 'name']].applymap(lower)
c360.sort_values(['Name'], inplace = True)
me.sort_values(['name'], inplace = True)
c360['website'] = c360['website'].apply(web)
me['website'] = me['website'].apply(web)
print('Halfway of data loading')
c360['Name_list'] = c360['Name'].apply(col_name)
me['name_list'] = me['name'].apply(col_name)
me['estb'] = me['year_of_establishment']
me.drop(['year_of_establishment'], axis = 1, inplace=True)
c360['Name']

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


NameError: name 'start' is not defined

In [7]:
c360.loc[1, ['Name', 'City', 'State', 'website']] = me.loc[1, ['name', 'city', 'state', 'website']].values

In [8]:
print('Mereexams\n', me.columns[:15])
print('C360\n', c360.columns)

Mereexams
 Index(['cid', 'name', 'address_line1', 'address_line2', 'city', 'state',
       'pin_code', 'area', 'constructed_area', 'website',
       'year_of_establishment', 'type', 'Affiliated_to_University',
       'playground', 'auditorium'],
      dtype='object')
C360
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Id', 'Name',
       'known_as', 'Estd', 'State', 'City', 'Address', 'ownership',
       'Approved_by', 'Affilate_to', 'contact', 'Rank', 'website', 'score',
       'facilities', 'exams', 'stream', 'faculty', 'facebook', 'twitter',
       'youtube', 'wiki'],
      dtype='object')


In [217]:
for index, row1 in tqdm(me.iterrows()):
    points = 0
    for index, row2 in c360.iterrows():
        (match, p)  = same(row1, row2)
        if match: 
            mapp[row1['name']] = row2['Name']
            break
        elif p > points:
            points = p
            name = row2['Name']
    if points > 10:
        remark[row1['name']] = name

50it [05:23,  6.46s/it]


In [218]:
for key in mapp.keys():
    print(key, '\t', mapp[key], '\n\n') 

secab inst., of bba/bca,  college, bijapur 	 secab institute of business administration, bijapur 




In [219]:
for key in remark.keys():
    print(key, '\t', remark[key], '\n\n') 

j.h.bhalodiya women's  college-rajkot (commerce & b.b.a.) 	 shri jh bhalodiya womens college, rajkot 


jss's bharatesh college of business administration,  belgaum 	 padmaraj ariga bharatesh college of business administration, belgaum 




In [145]:
remark["indo global college of engg. abhipur"]

'indo global college of architecture, mohali'

In [136]:
me[me['name'] == 'ggs college of modern technology, kharar']

,cid,name,address_line1,address_line2,city,state,pin_code,area,constructed_area,website,...,no_of_separate_rooms_for_girls,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell
288,C10350,"ggs college of modern technology, kharar","Chandigarh-Ludhiana Highway, Kharar",nan,"kharar, mohali",nan,140301,13.0,13877,www.ggssachdeva.com,...,1.0,False,False,False,31053.0,56.0,True,True,True,True


In [138]:
c360[c360['Name'] == 'ggs college of modern technology, mohali']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Id,Name,known_as,Estd,State,City,Address,...,website,score,facilities,exams,stream,faculty,facebook,twitter,youtube,wiki
352,127,127,2353,2353,"ggs college of modern technology, mohali",nan,2002.0,punjab,kharar,Chandigarh-Ludhiana Highway Kharar-140301 Di...,...,http://www.ggssachdeva.com/cmt/,nan,"['Boys Hostel', 'Girls Hostel', 'Gym', 'Librar...",['JEE Main'],eng,64.0,nan,nan,nan,https://en.wikipedia.org/wiki/Guru_Gobind_Sing...


In [149]:
mapp.keys()

dict_keys(['aman bhalla institute of engg. & tech, jhakolari, pathankot', 'ggs college of modern technology, kharar', 'rimt- institute of engg. and technology, mandi gobindgarh', 'sachdeva engg. college for girls vpo mohali', 'sri sai college of engg and technology , badhani'])

In [143]:
remark.keys()

dict_keys(['aman bhalla institute of engg. & tech, jhakolari, pathankot', 'ct institute of management & technology, shahpur', 'ct institute of technology, shahpur', 'gulzar school of management, vill libra', 'guru teg bahadur khalsa institute  of engg. & technology', 'indo global college of engg. abhipur', 'sachdeva engg. college for girls vpo mohali', 'sri sai college of engg and technology , badhani', 'swami parmanand engg. college'])

In [99]:
me[me['name'] == 'ct institute of management & technology, shahpur']

,cid,name,address_line1,address_line2,city,state,pin_code,area,constructed_area,website,...,no_of_separate_rooms_for_girls,solar_power_generation,connectivity_nkn,connectivity_nmeict,no_of_books,no_of_journals,campus_friendly,grievance_redressal_mechanism,vigilance_cell,opportunity_cell
335,C10416,"ct institute of management & technology, shahpur","VILLAGE SHAHPUR, P.O. UDOPUR, VIA JAMSHER KHAS...","VILLAGE SHAHPUR, P.O. UDOPUR, VIA JAMSHER KHAS...",jalandhar,nan,144020,3.0,10494,http://www.ctgroup.in,...,1.0,False,True,True,nan,nan,True,True,True,True


In [154]:
c360[c360['Name'] == 'yogi vemana university, kadapa']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Id,Name,known_as,Estd,State,City,Address,...,website,score,facilities,exams,stream,faculty,facebook,twitter,youtube,wiki
32,32,11,140,140,"yogi vemana university, kadapa",nan,2006.0,andhra pradesh,kopparthi,Kadapa – 516 003,...,http://www.yogivemanauniversity.ac.in/,nan,"['Boys Hostel', 'Girls Hostel', 'Gym', 'Librar...",[' IPU CET BJMC '],media,108.0,nan,nan,nan,https://en.wikipedia.org/wiki/Yogi_Vemana_Univ...
112,112,11,140,140,"yogi vemana university, kadapa",nan,2006.0,andhra pradesh,kopparthi,Kadapa – 516 003,...,http://www.yogivemanauniversity.ac.in/,nan,"['Boys Hostel', 'Girls Hostel', 'Gym', 'Librar...",[' IPU CET BJMC '],media,108.0,nan,nan,nan,https://en.wikipedia.org/wiki/Yogi_Vemana_Univ...


In [14]:
    c360 = pd.read_csv('../all_colleges_car360.csv').astype(str)
    me = pd.read_csv('../all_colleges_mereexams.csv').astype(str)[0:1000]
    c360[['State', 'City', 'Name']] = c360[['State', 'City', 'Name']].applymap(lower)
    me[['state', 'city', 'name']] = me[['state', 'city', 'name']].applymap(lower)
    c360.sort_values(['Name'], inplace = True)
    me.sort_values(['name'], inplace = True)
    c360['website'] = c360['website'].apply(web)
    me['website'] = me['website'].apply(web)
    print('Halfway of data loading')
    c360['Name'] = c360['Name'].apply(col_name)
    me['name'] = me['name'].apply(col_name)
    me['estb'] = me['year_of_establishment']
    me.drop(['year_of_establishment'], axis = 1, inplace=True)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Halfway of data loading


In [32]:
for id, clg in remark:
    print(me[me['cid'] == id]['name'].values, clg, end = '\n\n')

[list(["b.v.v.sangha's", 'basaveshwar', 'commercecollege', 'bagalkot'])] ['basaveshwar', 'commerce', 'college', 'bagalkot']

[list(['bldeas', 'comm', 'bhs', 'arts', 'tgp', 'sci', 'college', 'jamakhandi'])] ["bldea's", 'commerce', 'bhs', 'arts', 'tgp', 'science', 'college', 'jamkhandi']

[list(['bldeas', 'j', 's', 's', 'college', 'edu.', 'bijapur'])] ["bldea's", 'vp', 'dr', 'pg', 'halakatti', 'college', 'engineering', 'technology', 'bijapur']

[list(['first', 'grade', 'college', 'muddebihal'])] ['first', 'grade', 'college', 'ankola']

[list(['j.h.bhalodiya', "women's", 'collegerajkot', '(commerce', 'b.b.a.)'])] ['shri', 'jh', 'bhalodiya', 'womens', 'college', 'rajkot']

[list(["jss's", 'bharatesh', 'college', 'business', 'administration', 'belgaum'])] ['padmaraj', 'ariga', 'bharatesh', 'college', 'business', 'administration', 'belgaum']

[list(['secab', 'bba/bca', 'college', 'bijapur'])] ['secab', 'business', 'administration', 'bijapur']

[list(['shree', 'a.v.j.', 'oza', 'b.ed.', 'colle